In [1]:
from preprocess import preprocess
import numpy as np
import dicom
import glob
from matplotlib import pyplot as plt
import os
import cv2
import xgboost as xgb
import mxnet as mx
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def get_extractor():
    model = mx.model.FeedForward.load('model/resnet-50', 0, ctx=mx.gpu(), numpy_batch_size=1)
    fea_symbol = model.symbol.get_internals()["flatten0_output"]
    feature_extractor = mx.model.FeedForward(ctx=mx.gpu(), symbol=fea_symbol, numpy_batch_size=32,
                                             arg_params=model.arg_params, aux_params=model.aux_params,
                                             allow_extra_params=True)

    return feature_extractor 

MIN_BOUND = -1000.0
MAX_BOUND = 400.0
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

PIXEL_MEAN = 0.25

def zero_center(image):
    image = image - PIXEL_MEAN
    return image

def next_batch(sample_image):
    batch = []
    cnt = 0
    dx = 40
    ds = 512
    for i in range(0, sample_image.shape[0] - 3, 3):
        tmp = []
        for j in range(3):
            img = sample_image[i + j]
            img = 255.0 / np.amax(img) * img
            img = cv2.equalizeHist(img.astype(np.uint8))
            img = img[dx: ds - dx, dx: ds - dx]
            img = cv2.resize(img, (224, 224))
            tmp.append(img)

        tmp = np.array(tmp)
        batch.append(np.array(tmp))
    batch = np.array(batch)
    return batch

# def calc_features():
#     net = get_extractor()
#     for folder in glob.glob(path):
#         batch = next_batch(image)
#         feats = net.predict(batch)
#         np.save(os.path.join(BASE_PATH, folder), feats)
#         print folder

# INPUT_FOLDER = '/media/zzz/OS/stage1/'
# patients = os.listdir(INPUT_FOLDER)
# patients.sort()

In [20]:
file_path = '/home/zzz/Desktop/cogs181/npy_data'
save_path = '/media/zzz/New/pre_npy'
patients = os.listdir(file_path)
patients.sort()

In [21]:
net = get_extractor()
for i in range(len(patients)):
#    if not os.path.isfile(save_path+'/'+patients[i]):
    image = np.load(file_path+'/'+patients[i])
    image = normalize(image)
    image = zero_center(image)
    batch = next_batch(image)
    feats = net.predict(batch)
    np.save(os.path.join(save_path, patients[i]), feats)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.


In [15]:
a = 0
for i in range(0,1594):
    if os.path.isfile(file_path+'/'+patients[i]):
        a +=1
print a
print len(patients)

1594
1594


In [13]:
import os, urllib
def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.urlretrieve(url, filename)
def get_model(prefix, epoch):
    download(prefix+'-symbol.json')
    download(prefix+'-%04d.params' % (epoch,))

get_model('http://data.mxnet.io/models/imagenet/resnet/50-layers/resnet-50', 0)

In [14]:
import mxnet as mx
sym, arg_params, aux_params = mx.model.load_checkpoint('resnet-50', 0)

In [15]:
mod = mx.mod.Module(symbol=sym, context=mx.gpu())

In [16]:
mod.bind(for_training = False,
         data_shapes=[('data', (1,3,224,224))])
mod.set_params(arg_params, aux_params)

/home/zzz/mxnet/python/mxnet/module/base_module.py:64: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [22]:
image = np.load(file_path+'/'+patients[853])
print patients[853]

image = normalize(image)
image = zero_center(image)
batch = next_batch(image)
feats = net.predict(batch)

8a97ff581c17a49a3ef97144efde8a19.npy
